### Create

In [10]:
import pymongo

Mongo_url = 'mongodb://localhost:27017'
Base_datos = 'Horarios'
Coleccion = 'Empl'
empl_id = 391570


try:
    Cliente = pymongo.MongoClient(Mongo_url, serverSelectionTimeoutMS= 1000)
    horario_data = Cliente[Base_datos]
    empl_data = horario_data[Coleccion]
    Empleado = {
        'nro_empl': 391570,
        'meses' : {'Abril':[]},
        'jornada': '12:00:00'
    }
    result = empl_data.insert_one(Empleado)
    Cliente.close()
except pymongo.errors.ServerSelectionTimeoutError as errorTiempo:
    print("Tiempo excedido de carga")
except pymongo.errors.ConnectionFailure as errorConexion:
    print("Fallo al conectarse a mongodb"+errorConexion)

### DROP

In [9]:
import pymongo

Mongo_url = 'mongodb://localhost:27017'
Base_datos = 'Horarios'
Coleccion = 'Empl'
empl_id = 391570

Cliente = pymongo.MongoClient(Mongo_url, serverSelectionTimeoutMS= 1000)
horario_db = Cliente[Base_datos]
empl_colecc = horario_db[Coleccion]

try:
    result= empl_colecc.drop()
    Cliente.close()
except pymongo.errors.ServerSelectionTimeoutError as errorTiempo: # type: ignore
    print("Tiempo excedido de carga")
except pymongo.errors.ConnectionFailure as errorConexion: # type: ignore
    print("Fallo al conectarse a mongodb"+errorConexion)

### Ventana Calculador

In [13]:
from tkinter import*
from tkinter import ttk
from tkinter import messagebox
import pymongo
import datetime

Mongo_url = 'mongodb://localhost:27017'
Base_datos = 'Horarios'
Coleccion = 'Empl'


Cliente = pymongo.MongoClient(Mongo_url, serverSelectionTimeoutMS= 1000)
horario_db = Cliente[Base_datos]
Empl_colecc = horario_db[Coleccion]


mes = 'Abril'
node = f'meses.{mes}'
nro = 391570
horas= {}

# Funciones para formatear str a horas o int para operar

#   convertir: toma el formato str hh:mm-hh:mm o el formato hh:mm-hh:mm/hh:mm-hh:mm para jornadas de partido
#              y calcula el total de horas de la jornada diaria

def convertir(hr):
    if '/' in hr: 
        inicio1 = datetime.datetime.strptime(hr[0:5],'%H:%M')
        final1 = datetime.datetime.strptime(hr[6:11], '%H:%M')
        inicio2 = datetime.datetime.strptime(hr[12:17], '%H:%M')
        final2 = datetime.datetime.strptime(hr[18:], '%H:%M')
        return (final1-inicio1) + (final2-inicio2)
    else:
        inicio = datetime.datetime.strptime(hr[0:5],'%H:%M')
        final = datetime.datetime.strptime(hr[6:], '%H:%M')
        return (final-inicio)


#   extras: toma el formato str hh:mm:ss tanto de la jornada contratada como del total de horas 
#           semanales y lo transforma para poder restar y devolver las horas extras realizadas 
#           en la semana
    

def extras(jornada,horas_total):
    j = jornada.split(':')
    h = horas_total.split(':')
    H = int(h[0])-int(j[0])
    M = int(h[1])-int(j[1])
    return f'{H}:{M:02d}:00'


# Funciones para mostrar los datos en TK Inter

# rellenar: nos crea un diccionario temporal mientras se rellenan los horarios trabajados
#           de cada de día de la semana, se hace de esta manera para que al guardarlo
#           en el documento json toda la semana sea un elemento único en la lista 
#           del mes correspondiente


def rellenar():
    if len(horario.get()) != 0 and len(dia.get()) != 0:
        horas[dia.get()]=[horario.get(),str(convertir(horario.get()))]
        dia.delete(0,END)
        horario.delete(0,END)



# actualizar: una vez completada la semana tendremos el botón de confirmar horario que nos
#             actualizará el documento json con los datos de la semana y creará 
#             el total de la semana y las horas extras realizadas 
#             como últimos elementos del diccionario 


def actualizar():
    global horas
    sumatoria = [convertir(s[0]) for k,s in horas.items() if s[0] != '']
    htotales= sum(sumatoria,datetime.timedelta())
    H = int(htotales.total_seconds()//3600)
    M = int(htotales.total_seconds()%3600//60)
    time = f'{H}:{M:02d}:00'
    horas['Total Semana'] = ['', time]
    for jornada in Empl_colecc.find({'nro_empl': nro}):
        J = (jornada['jornada'])
    h_extras = extras(J,time)
    horas['Horas Extras'] = ['',h_extras]
    Empl_colecc.update_one({'nro_empl': nro},{'$push':{node:horas}}) 
    horas = {}
    mostrardatos() 


# mostrardatos: es la funcion que utilizamos para insertar los elementos del documento json en nuestra
#               tabla de tk inter. Esta función se encuentra también está metida dentro de las funciones
#               que actualizan datos como -actualizar- y -cambiar- para que no se dupliquen
#               los elementos mostrados en la tabla


def mostrardatos():
    registros=tabla.get_children()
    for registro in registros:
        tabla.delete(registro)
    for empleado in Empl_colecc.find({'nro_empl': nro}):
        for i in reversed(range(len(empleado['meses'][mes]))):
            for d,h in reversed((empleado['meses'][mes][i].items())):
                tabla.insert('',0,text='',values=(d,h[0],h[1]))
            tabla.insert('',0,text='',values=('','',''))


# cambiar:  recogerá los datos necesarios para cambiar de mes, jornada 
#           o de documento de empleado si existiera


def cambiar():
    global mes
    global node
    global nro
    
    if len(month.get()) != 0:
        mes = month.get()
        node = f'meses.{mes}'
        month.delete(0,END)
    if len(e_id.get()) != 0:
        nro = e_id.get()
        e_id.delete(0,END)
    if len(jornada.get()) != 0:
        Empl_colecc.update_one({'nro_empl': nro},{'$set':{'jornada':f'{jornada.get()}:00:00'}})
        jornada.delete(0,END) 
    mostrardatos()



# Creación y formateo de la ventana de tk inter 

ventana = Tk()
ventana.geometry('500x350')
ventana.title('Horario de Pull & Bear')

tabla = ttk.Treeview(ventana, columns=['#1','#2','#3'], show='headings')
tabla.grid(row=2,column=0,columnspan=4)
tabla.heading('#1',text='Día')
tabla.heading('#2',text='Horario')
tabla.heading('#3',text='Total Horas')

tabla.column('#1', width=150, anchor=CENTER)
tabla.column('#2', width=200, anchor=CENTER)
tabla.column('#3', width=150, anchor=CENTER)
tabla.column('#0', width=0, anchor=CENTER)

#  Labels y entries de la derecha

Label(ventana,text='Dia:').place(x=292,y=235)
dia=Entry(ventana)
dia.place(x=330,y=235)

Label(ventana,text='Horario:').place(x= 270,y=258)
horario=Entry(ventana)
horario.place(x=330,y=260)


#  Labels y entries de la izquierda


Label(ventana,text='# Empleado:').place(x=10,y=235)
e_id = Entry(ventana)
e_id.place(x=100,y=235)

Label(ventana,text='Mes:').place(x=50,y=263)
month = Entry(ventana)
month.place(x=100,y=263)

Label(ventana,text='Jornada:').place(x=30,y=287)
jornada = Entry(ventana)
jornada.place(x=100,y=287)

#  Botones de la derecha


crear=Button(ventana,text='Guardar horas', command=rellenar, fg='white',bg='#B4B0FE')
crear.place(x=350,y=285)

crear1=Button(ventana,text='Confirmar Semana', command=actualizar, fg='white',bg='#B4B0FE')
crear1.place(x=340,y=315)

#  Boton izquierdo

modif = Button(ventana,text='Modificar Datos', command=cambiar, bg='#B4B0FE') 
modif.place(x=114,y=310)

mostrardatos()
ventana.mainloop()
